stable diffusion

In [ ]:
import os
import itertools
from IPython.display import Image, display
from diffusers import StableDiffusionPipeline  # Ensure you have the correct import for your pipeline
import torch

pool_keywords = [
    "hotel pool area with chairs", "hotel pool area with people",
    "hotel pool area with lights direction from north", " man sitting on hotel pool lounge"," man sitting on hotel pool chair"
]

# Define a function to generate prompts using combinations of keywords
def generate_prompts(keywords, num_words=3):
    prompts = []
    for i in range(1, num_words + 1):
        keyword_combinations = itertools.combinations(keywords, i)
        for combination in keyword_combinations:
            prompt = "Generate an image of a " + ", ".join(combination) + "."
            prompts.append(prompt)
    return prompts

# Function to generate annotations for a given image and prompt
def generate_annotations(image_path, chairs_info, light_direction, prompt):
    annotations = {
        "image_path": image_path,
        "chairs": chairs_info,
        "light_direction": light_direction,
        "prompt": prompt  # Include the prompt in the annotations
    }
    return annotations

def generate_images(
    prompts,
    num_images_to_generate,
    num_images_per_prompt=4,
    guidance_scale=8,
    output_dir="/content/drive/My Drive/stable_difuser_images",
    display_images=False,
):

    num_iterations = num_images_to_generate // num_images_per_prompt
    os.makedirs(output_dir, exist_ok=True)

    # Initialize pipeline
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)

    count = 0
    for prompt in prompts:
        for _ in range(num_iterations):
            images = pipeline(prompt, num_images_per_prompt=num_images_per_prompt, guidance_scale=guidance_scale)
            for idx, image in enumerate(images.images):
                count += 1
                image_name = f"{output_dir}/image_{count}.png"
                image.save(image_name)
                if display_images:
                    display(Image(filename=image_name, width=1000, height=1000))

# Generate prompts
prompts = generate_prompts(pool_keywords)

# Generate images using the prompts
generate_images(
    prompts=prompts,
    num_images_to_generate=30,  # Change this to the desired number of images
    num_images_per_prompt=4,
    guidance_scale=8,
    output_dir="/content/drive/My Drive/stable_difuser_images",
    display_images=False
)
